<h1> Using Labeled Data to Train a Network, Use The Network to Label New Videos, and Create Trajectories </h1> 
<p> Use another notebook create a project, extract frames, and label training/testing data </p>

This notebook illustrates how to:
- train a network
- evaluate a network
- analyze a novel video

many of the functions have additional parameters / complexity, see the DLC docs for more inf on each.

This assumes you already have a project folder with labeled data! 

In [ ]:
# make sure you graphic driver is accessable
!nvcc --version

In [ ]:
# check the tensorflow version
import tensorflow as tf
tf.__version__

<h1> The following is very important as it allows the current GPU process to grow dynamically </h1>
Without this option tf will likely run out of VRAM when trying to update the weight tensor. In theory, these options could cause the GPU to run out of memory entirely, but there is no other way to allow training to complete successfully. 

In [ ]:
#allow video memory growth as network expands to avoid convolutional network errors
TF_FORCE_GPU_ALLOW_GROWTH = True
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
#let's make sure we see a GPU:
#tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

## Start here for training DeepLabCut and analyzing new videos!
<p><br>If the first imports fail, there is again - sadly - an issue with you enviroment. Make sure all packages beside DLC are installed via conda. </p> 

In [ ]:
#GUIs don't work on in Docker (or the cloud), so label your data locally on your computer! 
#This notebook is for you to train and run video analysis!
import os

In [ ]:
# now we are ready to train!
#should see version 2.0.8
import deeplabcut
deeplabcut.__version__

<h1> change to your path: </h1>
<p> this should be the same path as the one in the createDLCproject notebook. The path is the path to the config.yaml file, not the project directory itself </p> 

In [ ]:
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam1-spencerloggia-2021-04-04\config.yaml'

## Create a training dataset
This function generates the training data information for DeepCut (an hdf5 file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles.

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

Now it is the time to start training the network!

## Start training
This function trains the network for a specific shuffle of the training dataset. Set `gputouse` if you want to use a specific hardware accelerator

In [ ]:
#reset in case you started a session before...
#tf.reset_default_graph()

deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=100, gputouse=0)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.3M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....


### A couple tips for possible troubleshooting (1): 

if you get **permission errors** when you run this step (above), first check if the weights downloaded. They should be under 'init_weights' (see path in the pose_cfg.yaml file). You can enter the DOCKER in the terminal:

### Troubleshooting (2): 
if it appears the training does not start (i.e. "Starting training..." does not print immediately),
then you have another session running on your GPU. Go check "nvidia-smi" and look at the process names. You can only have 1 per GPU!)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(path_config_file)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

## There is an optional refinement step
- if you see higher pixel errors, check the labelled videos before worrying. It's possible that misclassified points when the mouse is not in the frame are driving up error.
- if your pixel errors are not low enough, use DLC docs on how to refine yur network!
- You will need to adjust the labels (you can use the createDLCproject notebook)
-  see DLC protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

Keep in mind that generating the 2D trajectories here is not required for 3D labelling. I recommend not 2d labelling all the data, maybe just a few examples for verification.
The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
# adjust path to where raw vids are located. 2D label h5 files are generated in same directory.
VIDEO_SOURCE = r"F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label"
video_all = os.listdir(VIDEO_SOURCE)
video = []
for i in range(0,len(video_all)):
    video.append(VIDEO_SOURCE + '\\' + video_all[i])
    
deeplabcut.analyze_videos(path_config_file,video)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam1-spencerloggia-2021-04-04\config.yaml'
deeplabcut.create_labeled_video(path_config_file,video)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
#for making interactive plots.
#deeplabcut.plot_trajectories(path_config_file,videofile_path, plotting=True)

deeplabcut.plot_trajectories(path_config_file,video,showfigures=True)